In [1]:
import re
from sklearn.neighbors import *
from sklearn.ensemble import *
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from load_data import get_data

# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier

In [2]:
x_train, y_train, x_pred, session_ids = get_data(is_fill=True)

/projects/ADA/load_data.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  alldf = pd.concat([train_df,test_df])


Training set size: (24584, 63)


In [ ]:
%%time
min_impurity_split = [1e-5, 1e-6, 1e-7, 1e-8, 1e-9]
param_grid = {
              'max_iter': randint(10, 1000),
              'max_depth': randint(2, 10),
              'min_samples_leaf': randint(2, 40),
             }
clf = RandomizedSearchCV(HistGradientBoostingClassifier(),
                   scoring='roc_auc',
                   param_distributions=param_grid, cv=2, n_jobs=4, n_iter=1500,
                   verbose=5)
clf.fit(x_train.to_numpy(), y_train)

Fitting 2 folds for each of 1500 candidates, totalling 3000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   31.3s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  5.4min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  9.4min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 14.5min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed: 20.4min
[Parallel(n_jobs=4)]: Done 874 tasks      | elapsed: 28.2min


In [4]:
i = clf.cv_results_['mean_test_score'].argmax()

In [6]:
clf.cv_results_['mean_test_score'].max()

0.8848658987005172

In [7]:
print(f"max_iter={clf.cv_results_['param_max_iter'][i]}, max_depth={clf.cv_results_['param_max_depth'][i]}, min_samples_leaf={clf.cv_results_['param_min_samples_leaf'][i]}")

max_iter=60, max_depth=7, min_samples_leaf=37


In [ ]:
%%time
param_grid = {'l2_regularization':[10],'max_iter': [50, 100, 200, 300]}
clf = GridSearchCV(HistGradientBoostingClassifier(),
                   scoring='roc_auc',
                   param_grid=param_grid, cv=2, n_jobs=8,
                   verbose=5, return_train_score=True)
clf.fit(x_train.to_numpy(), y_train.to_numpy())

In [ ]:
plt.plot(clf.cv_results_['mean_test_score'])
plt.plot(clf.cv_results_['mean_train_score'])
plt.show()